In [25]:
import numpy as np
import scipy.stats as scs
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import matplotlib as mpl
import datetime as dt
import typing
import scipy.stats as stats
from datetime import datetime, timedelta
plt.style.use('seaborn')
%matplotlib inline
import sympy

In [26]:
google = pd.DataFrame(yf.download('googl', start = '2022-12-01')['Adj Close']).pct_change().dropna()

[*********************100%***********************]  1 of 1 completed


In [27]:
d = pd.date_range(start='2023-02-02', end = '2023-02-17', freq='D')
len(d)

16

In [7]:
start = '2023-02-02'
end = '2023-02-17'
date_range = pd.date_range(start = start, end = end , freq = 'B')
delta = date_range[-1] - date_range[0]
print("Time delta:", delta)

Time delta: 15 days 00:00:00


# Option Formula

## Call Option 

- Gives the owner the right to buy but not the obligation to buy the underlying asset at a specific price $K$.

$C_{0,t} = S_0 \text{N}({d_1}) - Ke^{-rT} \text{N}({d_2})$

## Put Option

$P_{0,t} = Ke^{-rT} \text{N}({-d_2}) - S_0 \text{N}({-d_1})$


In [43]:
def call(S0, K, r, sigma, T):
    """
    Returns the Black-Scholes Call Option Price
    S0: Spot price at time 0
    K: Strike price
    r: risk-free interest rate
    sigma: annualized volatility of the option
    T: Time to maturity

    """
    d1 = (np.log(S0/K)+(r+((sigma**2)/2))*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    pv = K*np.exp(-r*T)

    C = S0*(stats.norm.cdf(d1)) - pv*(stats.norm.cdf(d2))
    delta = stats.norm.cdf(d1)
    theta = -r*K*np.exp(-r*T)*stats.norm.cdf(d2) - ((sigma*S0)/(2*np.sqrt(T)))*stats.norm.pdf(d1)
    gamma = stats.norm.pdf(d1)/((sigma*S0*np.sqrt(T)))
    vega = S0*np.sqrt(T)*stats.norm.pdf(d1)
    DF = pd.DataFrame([np.round(C,4)], columns = ['Call Option'],index = ['Price'])
    DF.loc['Delta'] = delta 
    DF.loc['Theta'] = theta
    DF.loc['Gamma'] = gamma
    DF.loc['Vega'] = vega
    
    return DF
def put(S0, K, r, sigma, T):
    """
    Returns the Black-Scholes Put Option Price
    S0: Spot price at time 0
    K: Strike price
    r: risk-free interest rate
    sigma: annualized volatility of the option
    T: Time to maturity

    """
    d1 = (np.log(S0/K)+(r+((sigma**2)/2))*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    pv = K*np.exp(-r*T)

    C = pv*(stats.norm.cdf(-d2)) - S0*(stats.norm.cdf(-d1))
    delta = -stats.norm.cdf(-d1)
    theta = r*K*np.exp(-r*T)*stats.norm.cdf(-d2) - ((sigma*S0)/(2*np.sqrt(T)))*stats.norm.pdf(d1)
    gamma = stats.norm.pdf(d1)/((sigma*S0*np.sqrt(T)))
    vega = S0*np.sqrt(T)*stats.norm.pdf(d1)
    DF = pd.DataFrame([np.round(C,4)], columns = ['Put Option'],index = ['Price'] )
    DF.loc['Delta'] = delta
    DF.loc['Theta'] = theta
    DF.loc['Gamma'] = gamma
    DF.loc['Vega'] = vega
    return DF

In [44]:
call(S0=96,K= 100,r = 0.1,sigma= .4,T = (5/365))

,Call Option
Price,0.510700
Delta,0.206311
Theta,-48.715500
Gamma,0.063458
Vega,3.204519


print(call.__doc__)

In [34]:
poss = [-1000,-500,-2000,-500]
gamma = [2.2,.6,1.3,1.8]
delta = [.5,.8,-.4,.7]
vega = [1.8,.2,.7,1.4]
V_g = np.dot(poss,gamma)
V_d = np.dot(poss,delta)
V_v = np.dot(poss,vega)
print(f'Portfolio Gamma is {V_g}; Portfolio delta is {V_d}; Portfolio vega is {V_v}.')

Portfolio Gamma is -6000.0; Portfolio delta is -450.0; Portfolio vega is -4000.0.


In [41]:
A = sympy.Matrix([[1,.6,.1],[0,1.5,.5],[0,.8,.6]])
b = sympy.Matrix([450,6000,4000])
A.solve(b)

Matrix([
[-1710.0],
[ 3200.0],
[ 2400.0]])

In interactive brokers I currently own two put contracts on Mastercard. 


In [49]:
A_1 = sympy.Matrix([[1,63.3],[0,1.1]])
b_2 = sympy.Matrix([76,-2.2])

A_1.solve(b_2)

Matrix([
[202.6],
[ -2.0]])

In [61]:
A_2 = sympy.Matrix([[1,56.4,-43.6],[0,1.6,1.8],[0,75.2,73.8]])
b_3 = sympy.Matrix([0,0,1000])
A_2.solve(b_3)

Matrix([
[-9912.03703703703],
[ 104.166666666667],
[-92.5925925925925]])

In [73]:
A_3 = sympy.Matrix([[56.4,-43.6],[75.2,73.8]])
b_4 = sympy.Matrix([0,175])
A_3.solve(b_4)

Matrix([
[1.02539429972154],
[1.32642748863062]])

In [74]:
call(49,50,.05,.2,0.3846)

,Call Option
Price,2.400500
Delta,0.521602
Theta,-4.305390
Gamma,0.065545
Vega,12.105243


In [51]:
call(56,57,.05,.3,.25)

,Call Option
Price,3.212700
Delta,0.516087
Theta,-7.981188
Gamma,0.047455
Vega,11.161300


In [52]:
call(54,57,.05,.3,.1666)

,Call Option
Price,1.620200
Delta,0.377409
Theta,-8.478146
Gamma,0.057462
Vega,8.374592


In [13]:
call(100,105,.04,.3,.25)

,Call Option
Price,4.321500
Delta,0.427163
Theta,-0.052794
Gamma,0.026152
Vega,19.613729


In [14]:
put(100,110,.04,.15,.25)

,Put Option
Price,9.403700
Delta,-0.864327
Theta,0.565564
Gamma,0.029048
Vega,10.892980


In [15]:
print(call.__doc__)


    Returns the Black-Scholes Call Option Price
    S0: Spot price at time 0
    K: Strike price
    r: risk-free interest rate
    sigma: annualized volatility of the option
    T: Time to maturity

    


In [16]:
def option(type = None, So = None, K = None, r = None, sigma = None, current_date = None, expiration_date = None):
    no_business_days = np.busday_count(begindates = current_date, enddates = expiration_date)
    Time = (no_business_days/252)
    # Divide by 252 because there are 252 trading days
    d1 = (np.log(So/K)+(r+((sigma**2)/2))*Time)/(sigma*np.sqrt(Time))
    d2 = d1 - sigma*np.sqrt(Time)
    present_value = K*(np.exp(-r*(Time)))
    if type == 'Call':
        price = So*(scs.norm.cdf(d1))-present_value*(scs.norm.cdf(d2))
    elif type == "Put":
        price = present_value*(scs.norm.cdf(-d2))-So*(scs.norm.cdf(-d1))
    else:
        print("Wrong security name")
    return pd.DataFrame([np.round(price,4)],columns = ['Option Price'])
        
        

In [17]:
option('Put',So = 444.49,K = 445, r = 0.05388,sigma=.1547,current_date='2023-08-11',expiration_date='2023-10-20')

,Option Price
0,10.1643


In [18]:
def fx_options(spot_fx_rate: float,strike_fx_rate:float,sigma: float ,domestic_interest: float,foreign_interest:float,domestic_currency:str, foreign_currency:str, spot_date:str,expiration_date: str, option_type:str):
    tau = len(pd.date_range(start=spot_date,end = expiration_date))
    if domestic_currency == 'USD':
        t1 = tau/360
    else:
        t1 = tau/365
    if foreign_currency == 'USD':
        t2 = tau/360
    else:
        t2 = tau/365
    domestic_discount = np.exp(-domestic_interest*t1)
    foreign_discount = np.exp(-foreign_interest*t2)
    d1 =(np.log(spot_fx_rate/strike_fx_rate)+((domestic_interest-foreign_interest)+((sigma**2)/2))*tau)/(sigma*np.sqrt(tau))
    d2 = d1 - sigma*np.sqrt(tau)

    if option_type == 'Call':
        price = spot_fx_rate*foreign_discount*(scs.norm.cdf(d1)) - strike_fx_rate*domestic_discount*(scs.norm.cdf(d2))

    elif option_type == 'Put':
        price = strike_fx_rate*domestic_discount*(scs.norm.cdf(-d2)) -spot_fx_rate*foreign_discount*(scs.norm.cdf(-d1))
    else:
        print('Wrong Option Type')

    return np.round(price,6)


    


In [19]:
fx_options(1.6,1.170,0.05,.0475,.0475,'EUR','GBP','2023-02-27','2023-02-28','Call')

0.429888

In [20]:
scs.norm.cdf(0)

0.5

In [21]:
option('Call', So = 106.7,K = 95,r = .04,sigma = .5, current_date='2023-02-02',expiration_date='2023-02-17')

,Option Price
0,12.5464


In [22]:
option('Call', So = 112,K = 95,r = .0468,sigma = .5, current_date='2023-02-03',expiration_date='2023-02-17')

,Option Price
0,17.3773


In [23]:
scs.norm.cdf(-.206)

0.4183954556526073